In [3]:
import numpy as np

from keras.layers import Dense, Dropout, Input
from keras.layers import Conv2D, MaxPooling2D,Flatten
from keras.layers.merge import concatenate
from keras.models import Model
from keras.utils import to_categorical,plot_model
from keras.datasets import mnist


In [4]:
(x_train, y_train),(x_test,y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [5]:
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [8]:
img_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1,img_size,img_size,1])
x_test = np.reshape(x_test,[-1,img_size,img_size,1])
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [10]:
input_shape = (img_size,img_size,1)
batch_size = 32
kernel_size = 3
dropout = 0.4
n_filters = 32

In [14]:
#Y-Network left
left_input = Input(shape = input_shape)
x = left_input
filters = n_filters

In [15]:
# Conv2D-Dropout-MaxPolling2D -> 3-layers
# 계층 지날 때 마다 filter는 두배

for idx in range(3):
    x = Conv2D(
        filters = filters,
        kernel_size = kernel_size,
        padding = 'same',
        activation= 'relu'
              )(x)
    x = Dropout(dropout)(x)
    x = MaxPooling2D()(x)
    filters = filters*2
    print(x)

KerasTensor(type_spec=TensorSpec(shape=(None, 14, 14, 32), dtype=tf.float32, name=None), name='max_pooling2d_5/MaxPool:0', description="created by layer 'max_pooling2d_5'")
KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 64), dtype=tf.float32, name=None), name='max_pooling2d_6/MaxPool:0', description="created by layer 'max_pooling2d_6'")
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 3, 128), dtype=tf.float32, name=None), name='max_pooling2d_7/MaxPool:0', description="created by layer 'max_pooling2d_7'")


In [16]:
#Y-Network rigth

right_inputs = Input(shape = input_shape)
y = right_inputs
filters = n_filters

In [19]:
# Conv2D-Dropout-MaxPolling2D -> 3-layers
# 계층 지날 때 마다 filter는 두배

for idx in range(3):
    y = Conv2D(
        filters = filters,
        kernel_size = kernel_size,
        padding = 'same',
        activation = 'relu'
    )(y)
    y = Dropout(dropout)(y)
    y = MaxPooling2D()(y)
    fiters = filters*2

In [20]:
# merge left & right

y = concatenate([x,y])


In [21]:
y = Flatten()(y)
y = Dropout(dropout)(y)
outputs = Dense(num_labels,activation = 'softmax')(y)


In [34]:
model = Model([left_input,right_inputs],outputs)

In [35]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 28, 28, 32)   320         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 28, 28, 32)   320         input_3[0][0]                    
___________________________________________________________________________________________

In [36]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics =['accuracy']
)

In [ ]:
#Y_network이므로 input -> 2
model.fit(
    [x_train,x_train],
    y_train,
    validation_data=([x_test,x_test],y_test),
    epochs = 20,
    batch_size = batch_size
)

Epoch 1/20
1875/1875 [==============================] - 84s 44ms/step - loss: 0.4191 - accuracy: 0.8610 - val_loss: 0.1970 - val_accuracy: 0.9807
Epoch 2/20
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0809 - accuracy: 0.9739 - val_loss: 0.1153 - val_accuracy: 0.9898
Epoch 3/20
1875/1875 [==============================] - 83s 44ms/step - loss: 0.0653 - accuracy: 0.9804 - val_loss: 0.0579 - val_accuracy: 0.9914
Epoch 4/20
1875/1875 [==============================] - 83s 44ms/step - loss: 0.0551 - accuracy: 0.9826 - val_loss: 0.0449 - val_accuracy: 0.9926
Epoch 5/20
1875/1875 [==============================] - 84s 45ms/step - loss: 0.0462 - accuracy: 0.9860 - val_loss: 0.0624 - val_accuracy: 0.9922
Epoch 6/20
1875/1875 [==============================] - 85s 45ms/step - loss: 0.0433 - accuracy: 0.9863 - val_loss: 0.0463 - val_accuracy: 0.9920cy: 0.98
Epoch 7/20
1875/1875 [==============================] - 85s 45ms/step - loss: 0.0430 - accuracy: 0.9869 - val_loss: 

Code review
-----------
   정확도에서 큰 개선이 일어나지는 않지만 Y_Network를 사용함으로써 두배의 parameter 양을 채워주어야 한다.   
   따라서 사용의 정의를 잘 내리는 것도 중요하다.
        